In [1]:
import openai
import llm2geneset
import pandas as pd
import random

aclient = openai.AsyncClient()

In [2]:
al002 = pd.read_csv("data/Stats.TECAN.AL002_vs_isotype.txt", sep="\t")
al002 = al002[al002["PercentDetectedSamples"] > 0.9]

In [3]:
al002_sig = al002[(al002["padj"] < 0.1) ]
genes = al002_sig["gene"].to_list()

In [4]:
random.seed(138337)
random.shuffle(genes)

In [5]:
print("\n".join(genes))

LILRB2
MX1
GSN
MFGE8
CCL3
GPD1
TRPV2
DHCR24
TNFAIP3
MATK
CXCL8
CHIT1
TPRKB
RASSF4
SMC6
FOS
RHOBTB2
RNF19B
FBP1
SLC11A1
DCSTAMP
HSD3B7
OAS2
TGM2
MCEMP1
FAM20C
ACP5
CD9
AP5B1


In [6]:
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", context=context_str,
                                                                 n_pathways=100)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:28<00:00, 28.69s/it]


In [7]:
df = llm2geneset_no_context_res[0]["ora_results"]
df.to_csv("outputs/no_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
25,Osteoclast differentiation,0.103448,0.001612,0.093750,64.157328,0.000014,0.001277,"FOS,ACP5,DCSTAMP","SRC,IL1B,ITGAV,TNF,TNFRSF11B,IL6,PTK2B,TRAF6,A...",29,32,3,158,434
20,Osteoclast differentiation and activation,0.103448,0.002570,0.058824,40.255578,0.000056,0.002631,"FOS,ACP5,DCSTAMP","JUN,NFKB1,SRC,TNFRSF11B,TRAF6,IKBKG,AKT1,FGFR3...",29,51,3,162,667


In [8]:
context_str = "in vitro microglia treated with a TREM2 agonist antibody"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              n_pathways=100)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:35<00:00, 35.16s/it]


In [9]:
df = llm2geneset_context_res[0]["ora_results"]
df.to_csv("outputs/with_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
37,Regulation of inflammatory signaling pathway,0.103448,0.002066,0.073171,50.074012,0.000029,0.002078,"FOS,TNFAIP3,CXCL8","CSF2,NFKB1,CHUK,IL1B,CCL2,TLR3,TNF,IL6,JAK2,CX...",29,41,3,160,368
84,Cellular response to TNF,0.103448,0.002368,0.063830,43.681585,0.000044,0.002078,"FOS,TNFAIP3,CXCL8","NFKB1,CHUK,IL1B,CCL2,TNF,IL6,CXCL8,BIRC2,IKBKB...",29,47,3,160,432


In [10]:
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(genes, set_descr, gene_sets)
df.to_csv("outputs/KEGG_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter
251,Rheumatoid arthritis,0.137931,0.004686,0.043011,29.434186,0.000010,0.003157,"FOS,CCL3,ACP5,CXCL8","CCL3L3,CXCL1,IL6,ATP6V1B2,HLA-DMA,TNFSF13,CTSK...",29,93,4
161,Measles,0.137931,0.007004,0.028777,19.693376,0.000048,0.007686,"FOS,MX1,OAS2,TNFAIP3","NFKB1,ADAR,IFNA5,IL6,TRAF6,IKBKB,IKBKG,PIK3CA,...",29,139,4
173,NOD-like receptor signaling pathway,0.137931,0.009120,0.022099,15.123643,0.000134,0.014265,"TRPV2,TNFAIP3,OAS2,CXCL8","VDAC1,NFKB1,ANTXR1,CXCL1,CARD17,IFNA5,NLRP1,IL...",29,181,4
136,IL-17 signaling pathway,0.103448,0.004736,0.031915,21.840792,0.000345,0.021238,"FOS,TNFAIP3,CXCL8","NFKB1,CXCL1,IL4,IL6,TRAF6,IKBKB,IKBKG,TRAF4,S1...",29,94,3
64,Coronavirus disease,0.137931,0.011690,0.017241,11.799049,0.000345,0.021238,"FOS,MX1,OAS2,CXCL8","NFKB1,RPL39,F13B,RPL7,IFNA5,ADAR,C5,RPL10,IL6,...",29,232,4
50,Chagas disease,0.103448,0.005140,0.029412,20.127789,0.000439,0.021238,"FOS,CCL3,CXCL8","CCL3L3,NFKB1,GNAQ,IL6,PLCB1,TRAF6,IKBKB,IKBKG,...",29,102,3
290,Toll-like receptor signaling pathway,0.103448,0.005240,0.028846,19.740716,0.000465,0.021238,"FOS,CCL3,CXCL8","CCL3L3,MAP2K3,NFKB1,CXCL9,IFNA5,IL6,TRAF6,IKBK...",29,104,3
191,Osteoclast differentiation,0.103448,0.006399,0.023622,16.165626,0.000832,0.033260,"FOS,LILRB2,ACP5","NFKB1,TRAF6,IKBKB,IKBKG,PPP3R1,PIK3CA,FYN,RELB...",29,127,3
